In [ ]:
%pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import sys

import boto3

In [3]:
client = boto3.client("bedrock-runtime")

In [29]:
prompt_data = """
On a given week, the viewers for a TV channel were
Monday: 6500 viewers
Tuesday: 6400 viewers
Wednesday: 6300 viewers


Question: How many viewers can we expect on Friday?
Answer: Based on the numbers given and without any more information, there is a daily decrease of 100 viewers. If we assume this trend will continue during the following days, we can expect 6200 viewers on the next day that would be Thursday, and therefore 6100 viewers on the next day that would be Friday.

Question: How many viewers can we expect on Saturday? (Think Step-by-Step)
Answer: ?

Solve the anwser
""" 

In [47]:
# Titan Text G1 - Lite: "amazon.titan-text-lite-v1",
# Titan Text G1 - Express: "amazon.titan-text-express-v1"

resp = client.invoke_model(
    modelId='amazon.titan-text-express-v1',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "inputText": prompt_data,
        "textGenerationConfig": {
            "maxTokenCount": 200,
            "temperature": 1.0
        }
    })
)

In [48]:
resp

{'ResponseMetadata': {'RequestId': 'b3afa945-2f9c-4adc-9407-645c805b4af1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 13 Jun 2025 06:33:32 GMT',
   'content-type': 'application/json',
   'content-length': '414',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b3afa945-2f9c-4adc-9407-645c805b4af1',
   'x-amzn-bedrock-invocation-latency': '5140',
   'x-amzn-bedrock-output-token-count': '131',
   'x-amzn-bedrock-input-token-count': '159'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x7fb31c399090>}

In [49]:
body = json.loads(resp.get("body").read())

body

{'inputTextTokenCount': 159,
 'results': [{'tokenCount': 131,
   'outputText': 'The viewers for a TV channel were Monday: 6500 viewers, Tuesday: 6400 viewers, Wednesday: 6300 viewers\nTuesday: 6400 viewers - 6500 viewers = -100 viewers\nThursday: 6200 viewers - 100 viewers = 6100 viewers\nFriday: 6100 viewers - 100 viewers = 6000 viewers\nSaturday: 6000 viewers + 100 viewers = 6100 viewers',
   'completionReason': 'FINISH'}]}

In [50]:
body['results'][0]['outputText']

'The viewers for a TV channel were Monday: 6500 viewers, Tuesday: 6400 viewers, Wednesday: 6300 viewers\nTuesday: 6400 viewers - 6500 viewers = -100 viewers\nThursday: 6200 viewers - 100 viewers = 6100 viewers\nFriday: 6100 viewers - 100 viewers = 6000 viewers\nSaturday: 6000 viewers + 100 viewers = 6100 viewers'